In [1]:
from pyscf import gto, scf, ao2mo, fci
import CC, Wick


In [2]:
bohr = 0.529177249

H2sep = 1.605 * bohr

mol = gto.Mole()
mol.verbose = 1
mol.atom = 'H 0 0 0; H 0 0 ' + str(H2sep)
mol.basis = 'sto-3g'
mol.spin = 0
mol.build()

Enuc = mol.energy_nuc()

mf = scf.ROHF(mol)
mf.kernel()

h1 = mf.mo_coeff.T.dot(mf.get_hcore()).dot(mf.mo_coeff)
eri = ao2mo.kernel(mol, mf.mo_coeff)

cisolver = fci.FCI(mol, mf.mo_coeff)

In [3]:
cisolver.kernel()

(-1.1284543355083052,
 array([[ 9.90656547e-01,  2.77555756e-17],
        [ 1.91460513e-17, -1.36380375e-01]]))

In [4]:
H2CCSDClusterOperator = CC.coupledCluster(mf, range(1,3))

2


In [5]:
print(H2CCSDClusterOperator)

-0.0
 + -4.394149808727142e-17 * a^{1\alpha}a_{0\alpha}
 + -4.394149808727142e-17 * a^{1\beta}a_{0\beta}
 + -0.06883332874447325 * a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + -0.06883332874447325 * a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}


In [6]:
waveOperator = CC.exponentialOperator(H2CCSDClusterOperator, 3)
hamiltonian = CC.get1bodyHamiltonian(mf) + CC.get2bodyHamiltonian(mf)
inverseWaveOperator = CC.exponentialOperator((-1) * H2CCSDClusterOperator, 3)

In [7]:
CCSDEnergy = Wick.vacuumExpectationValue(inverseWaveOperator * hamiltonian * waveOperator, [1,0]) + mol.energy_nuc()
print(CCSDEnergy)

-1.1284543355083052


In [10]:
print(CC.BCHSimilarityTransform(hamiltonian, CC.spinFreeNtupleExcitation((1,1), (0,0)), 4))

0.0
 + -4.7142356915405136e-17 * a^{1\alpha}a_{0\alpha}
 + -0.5337491016607854 * a^{1\alpha}a_{1\alpha}
 + -4.7142356915405136e-17 * a^{1\beta}a_{0\beta}
 + -0.5337491016607854 * a^{1\beta}a_{1\beta}
 + -1.1898506186070186 * a^{0\alpha}a_{0\alpha}
 + 2.600212552842601e-17 * a^{0\alpha}a_{1\alpha}
 + -1.1898506186070186 * a^{0\beta}a_{0\beta}
 + 2.600212552842601e-17 * a^{0\beta}a_{1\beta}
 + 0.0
 + 0.3226247132942424 * a^{0\alpha}a^{1\alpha}a_{1\alpha}a_{0\alpha}
 + 0.0
 + 4.163336342344337e-17 * a^{0\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + 0.3226247132942424 * a^{0\alpha}a^{1\beta}a_{1\beta}a_{0\alpha}
 + 2.7755575615628914e-17 * a^{0\alpha}a^{1\beta}a_{1\beta}a_{1\alpha}
 + 0.0
 + 0.0
 + 0.0
 + 0.0
 + 4.163336342344337e-17 * a^{0\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
 + 0.3226247132942424 * a^{0\beta}a^{1\alpha}a_{1\alpha}a_{0\beta}
 + 2.7755575615628914e-17 * a^{0\beta}a^{1\alpha}a_{1\alpha}a_{1\beta}
 + 0.0
 + 0.3226247132942424 * a^{0\beta}a^{1\beta}a_{1\beta}a_{0\beta}
 + 0.0
 +

In [11]:
print(hamiltonian)

-1.1898506186070186 * a^{0\alpha}a_{0\alpha}
 + 2.600212552842601e-17 * a^{0\alpha}a_{1\alpha}
 + -1.1898506186070186 * a^{0\beta}a_{0\beta}
 + 2.600212552842601e-17 * a^{0\beta}a_{1\beta}
 + -4.7142356915405136e-17 * a^{1\alpha}a_{0\alpha}
 + -0.5337491016607854 * a^{1\alpha}a_{1\alpha}
 + -4.7142356915405136e-17 * a^{1\beta}a_{0\beta}
 + -0.5337491016607854 * a^{1\beta}a_{1\beta}
 + 0.0
 + 0.0
 + 0.0
 + 0.0
 + 0.3270047556780619 * a^{0\alpha}a^{0\beta}a_{0\beta}a_{0\alpha}
 + 5.204170427930421e-17 * a^{0\alpha}a^{0\beta}a_{0\beta}a_{1\alpha}
 + 4.163336342344337e-17 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{0\alpha}
 + 0.09376099045135429 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}
 + 4.163336342344337e-17 * a^{0\alpha}a^{1\alpha}a_{0\alpha}a_{0\alpha}
 + 0.3226247132942424 * a^{0\alpha}a^{1\alpha}a_{1\alpha}a_{0\alpha}
 + 2.7755575615628914e-17 * a^{0\alpha}a^{1\alpha}a_{1\alpha}a_{1\alpha}
 + 4.163336342344337e-17 * a^{0\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + 0.3226247132942424 * a^{0